In [1]:
import pandas as pd
from dotenv import dotenv_values
from sqlalchemy import create_engine
import os
from tqdm import tqdm

In [2]:
df_country = pd.read_csv('../data/country_codes.csv', index_col = 'alpha2') #, skiprows=16, nrows=3308)
df_country

,name,alpha3,code,lat,lon
alpha2,,,,,
AF,Afghanistan,AFG,4,33.0000,65.0
AL,Albania,ALB,8,41.0000,20.0
DZ,Algeria,DZA,12,28.0000,3.0
AS,American Samoa,ASM,16,-14.3333,-170.0
AD,Andorra,AND,20,42.5000,1.6
...,...,...,...,...,...
WF,Wallis and Futuna,WLF,876,-13.3000,-176.2
EH,Western Sahara,ESH,732,24.5000,-13.0
YE,Yemen,YEM,887,15.0000,48.0


In [3]:
config = dotenv_values("../.env")

# define variables for the login
username = config['USER']
password = config['PASS']
host = config['HOST']
port = config['PORT']

In [4]:
url = f'postgresql://{username}:{password}@{host}:{port}/climate'
engine = create_engine(url, echo=False)

In [ ]:
with engine.begin() as conn:
    conn.execute("DROP TABLE IF EXISTS countries CASCADE;")
    conn.execute("""
        CREATE TABLE countries (
            --index INTEGER,
            alpha2 VARCHAR(2) PRIMARY KEY,
            name VARCHAR,
            alpha3 VARCHAR(3),
            code INTEGER,
            lat NUMERIC,
            lon NUMERIC
        );
    """)
    df_country.to_sql('countries', conn, if_exists='append', index=True)

In [5]:
filename = '../data/stations.txt'
# skiprows work with 16 and 17
df_station = pd.read_csv(filename, skiprows=16, encoding = 'unicode_escape')
df_station.columns= df_station.columns.str.strip().str.lower()
df_station.set_index('staid' , inplace=True)
df_station

,staname,cn,lat,lon,hght
staid,,,,,
1,VAEXJOE,SE,+56:52:00,+014:48:00,166
2,FALUN,SE,+60:37:00,+015:37:00,160
3,STENSELE,SE,+65:04:00,+017:09:59,325
4,LINKOEPING,SE,+58:24:00,+015:31:59,93
5,LINKOEPING-MALMSLAETT,SE,+58:24:00,+015:31:59,93
...,...,...,...,...,...
25153,BYDGOSZCZ-SZWEDEROWO,PL,+53:05:47,+017:59:42,55
25156,KROSNO,PL,+49:42:24,+021:46:09,326
25157,YLJA KRAFTVERK,NO,+61:11:49,+008:22:50,517


In [7]:
with engine.begin() as conn:
    conn.execute("DROP TABLE IF EXISTS stations  CASCADE;")
    conn.execute("""
        CREATE TABLE stations (
            staid INTEGER PRIMARY KEY,
            cn VARCHAR(2) REFERENCES countries(alpha2),
            staname VARCHAR,
            lat VARCHAR,
            lon VARCHAR,
            hght INTEGER
        );
    """)
    df_station.to_sql('stations', conn, if_exists='append', index=True)

In [3]:
def parse_file(filename):
    df = pd.read_csv(filename, skiprows=19, encoding = 'unicode_escape') #, index_col=0
    df.columns= df.columns.str.strip().str.lower()
    #df.set_index('staid' , inplace=True)
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
    df = df[df['q_tg']==0]
    df.drop(columns=['souid','q_tg'], inplace=True)
    return df

In [ ]:
x=0
with open("../data/mean_temperature.csv", mode="w", newline='') as file: # open
    for filename in tqdm(os.listdir("../data/")):
        # print (f'../data/test/{filename}')
        if 'TG_STAID' in filename and filename != 'TG_STAID022622.txt':  #file.startswith('TG_STAID') 'TG_STAID' in filename
            df = parse_file(f'../data/{filename}')
            x += len(df)
            df.to_csv(file, index=False, header=False)
x

In [6]:
# small table
x=0
with open("../data/mean_temperature_small.csv", mode="w", newline='') as file: # open
    for filename in tqdm(os.listdir("../data/")[0:12]):
        # print (f'../data/test/{filename}')
        if 'TG_STAID' in filename and filename != 'TG_STAID022622.txt':  #file.startswith('TG_STAID') 'TG_STAID' in filename
            df = parse_file(f'../data/{filename}')
            x += len(df)
            df.to_csv(file, index=False, header=False)
x

100%|██████████| 12/12 [00:02<00:00,  4.25it/s]


236320

In [ ]:
filename = '../data/test/TG_STAID000002.txt'
# skiprows work with 19 and 20
df_t = parse_file(filename)
df_t
#x= len(df_t)

